# 01 - Basic Usage of Qurry

In Qurry, we use the instance of `Qurry` to store quantum circuits, experiments, build experiment, and excute them on local simulator or pending to the real quantum machine on `IBM`. In this chapter (or we say this jupyter file), we will introduce how to use Qurry to build a simple quantum circuit and measure their Renyi Entropy on local simulator.


If you are using Colab, you can run the following command in terminal to install Qurry.

```bash
pip install -i https://test.pypi.org/simple/ qurry
```


In [1]:
from IPython.display import display
from qurry import __version__

__version__

'0.7.4.dev1'

## 1.1 Setup Environment and Creat experiment executor


In [2]:
from qurry import EntropyMeasure
from qiskit import QuantumCircuit

- Import simulator


In [3]:
from qiskit.providers.basic_provider import BasicProvider

basic_provider = BasicProvider()
backend_sim = basic_provider.get_backend("basic_simulator")

- Next, we will initialize our `EntropyMeasure` instance. There are two methods implemented based on two theories. The defaul method is `randomized` based on Randomized Measure. The other method is `hadamard` based on Hadamard Test. 

- In the following tutorial, we will use `randomized` as our measurement method.


In [4]:
experiment_hadamard = EntropyMeasure(method="hadamard")
print('| The first method is which is `Hadamard Test`".')
experiment_randomized = EntropyMeasure()
print("| The default method is which is `Randomized Measure` we will use.")

| The first method is which is `Hadamard Test`".
| The default method is which is `Randomized Measure` we will use.


---

## 1.2 Load quantum circuits

- In `Qurry`, there are built-in quantum circuits that construct certain wave functions, or equivalently, quantum states. They can be imported from `qurry.recipe`.
- The user can define customized circuits following Qiskit workflow.



In [5]:
from qurry.recipe import TrivialParamagnet, GHZ

In [6]:
sample01 = TrivialParamagnet(8)
print("| trivial paramagnet in 8 qubits:")
print(sample01)
sample02 = GHZ(8)
print("| GHZ in 8 qubits:")
print(sample02)
sample03 = QuantumCircuit(8)
sample03.x(range(0, 8, 2))
print("| Custom circuit:")
print(sample03)

| trivial paramagnet in 8 qubits:
     ┌───┐
q_0: ┤ H ├
     ├───┤
q_1: ┤ H ├
     ├───┤
q_2: ┤ H ├
     ├───┤
q_3: ┤ H ├
     ├───┤
q_4: ┤ H ├
     ├───┤
q_5: ┤ H ├
     ├───┤
q_6: ┤ H ├
     ├───┤
q_7: ┤ H ├
     └───┘
| GHZ in 8 qubits:
     ┌───┐                                   
q_0: ┤ H ├──■────────────────────────────────
     └───┘┌─┴─┐                              
q_1: ─────┤ X ├──■───────────────────────────
          └───┘┌─┴─┐                         
q_2: ──────────┤ X ├──■──────────────────────
               └───┘┌─┴─┐                    
q_3: ───────────────┤ X ├──■─────────────────
                    └───┘┌─┴─┐               
q_4: ────────────────────┤ X ├──■────────────
                         └───┘┌─┴─┐          
q_5: ─────────────────────────┤ X ├──■───────
                              └───┘┌─┴─┐     
q_6: ──────────────────────────────┤ X ├──■──
                                   └───┘┌─┴─┐
q_7: ───────────────────────────────────┤ X ├
                        

- After we prepare three quantum circuits, we append them to the `EntropyMeasure` object by the `.add()` method.
- The attribute `.waves` of `Qurry` object is a customized `dict`. we can check the stored circuits by printing the dictionary.

In [7]:
experiment_randomized.add(
    wave=sample01,  # The circuit object
    key="TrivialParamagnet",  # The name of the circuit
    replace=False,  # True for if you want to replace the circuit with the same name.
)  # Default is False, so if you add the same name when False,
# it will be added with serial number.
experiment_randomized.add(sample02, "GHZ")
experiment_randomized.add(sample03)
print(experiment_randomized.waves)

<WaveContainer={
    TrivialParamagnet: ...
    GHZ: ...
    2: ...
} with 3 waves load, a customized dictionary>


If you do not give the name of circuit, it will be named by series number.

Now, waves are loading, time to excute.


## 1.3 Excute Circuits


- We can use `.measure()` method to excute the specific circuit in the `EntropyMeasure` object. The `.measure()` method  returns a hash id of this experiment and save a `Experiment` object in `EntropyMeasure`.
- `EntropyMeasure.measure()` requires the following input parameters:
  - wave: the name of circuit in `Qurry.waves` object
  - times: the number of random measurements
  - shots: the number of shots for each measurement. The default is 1024.


In [8]:
exp1 = experiment_randomized.measure(
    wave="TrivialParamagnet", times=100, shots=1024, backend=backend_sim
)
exp1

'd355a026-ccd1-4150-b211-8af0efe11920'

- You can specify the backend by add a parameter `backend` in `measure` method, if you do not specify, it will use simulator as the default backend. Moreover, qiskit has multiple simulators, they are `QasmSimulatorPy` from `qiskit`, `AerSimulator` from `qiskit-aer`, and `AerSimulator` from `qiskit-aer-gpu`. Qurry will check the availability of these simulators and use the first available one as the default backend in following order:
  1. `AerSimulator` from `qiskit-aer-gpu`
  2. `AerSimulator` from `qiskit-aer`
  3. `QasmSimulatorPy` from `qiskit`


In [9]:
exp2 = experiment_randomized.measure(
    wave="TrivialParamagnet",
    times=100,
    shots=1024,
)
exp2

'89015bfb-3b78-43b4-9e1d-af57173c1627'

- And we can check the attribute `.exps` of `EntropyMeasure` object, it will return the hash id of all experiments.


In [10]:
print(experiment_randomized.exps)

<ExperimentContainer={
    d355a026-ccd1-4150-b211-8af0efe11920: ...
    89015bfb-3b78-43b4-9e1d-af57173c1627: ...
} with 2 experiments load, a customized dictionary>


- The following is the other informatiom you can access.


In [11]:
experiment_randomized.waves

<WaveContainer={
    TrivialParamagnet: ...
    GHZ: ...
    2: ...
} with 3 waves load, a customized dictionary>

In [12]:
experiment_randomized.exps[exp1]

<qurrentRandomized.Experiment with exp_id=d355a026-ccd1-4150-b211-8af0efe11920, EntropyRandomizedArguments(exp_name='exps', times=100, measure=(0, 8), unitary_loc=(0, 8), workers_num=16), Commonparams(exp_id='d355a026-ccd1-4150-b211-8af0efe11920', wave_key='TrivialParamagnet', shots=1024, backend=<qiskit.providers.basic_provider.basic_simulator.BasicSimulator object at 0x75725c4638d0>, run_args={}, transpile_args={}, tags=(), default_analysis=[], save_location=PosixPath('.'), filename='', files={}, serial=None, summoner_id=None, summoner_name=None, datetimes={'build': '2024-06-03 17:55:45', 'run': '2024-06-03 17:55:45'}), 0 unused arguments, 6 preparing dates, 2 experiment result datasets, and 0 analysis>

In [13]:
print(experiment_randomized.exps[exp1].statesheet())

 # qurrentRandomized.Experiment with exp_id=d355a026-ccd1-4150-b211-8af0efe11920
 - arguments
   - exp_name ---------------- exps
   - times ------------------- 100
   - measure ----------------- (0, 8)
   - unitary_loc ------------- (0, 8)
   - workers_num ------------- 16
 - commonparams
   - exp_id ------------------ d355a026-ccd1-4150-b211-8af0efe11920   # This is ID is generated by Qurry which is different from 'job_id' for pending.
   - wave_key ---------------- TrivialParamagnet
   - shots ------------------- 1024
   - backend ----------------- <qiskit.providers.basic_provider.basic_simulator.BasicSimulator object at 0x75725c4638d0>
   - run_args ---------------- {}
   - transpile_args ---------- {}
   - tags -------------------- ()
   - default_analysis -------- []
   - save_location ----------- .
   - filename ---------------- 
   - files ------------------- {}
   - serial ------------------ None
   - summoner_id ------------- None
   - summoner_name ----------- None
   - date

- Then, there is a experiment completed, and we can calculate the Renyi Entropy by `analyze`.


---

## 1.4 Post-Processing

- To obtain the Renyi Entropy of the whole system and the subsystems. We use `exps[].analyze()` method to calculate and store a `Analysis` object in `Experiment`.
- The `analyze` method requires the following input parameters:
  - degree: The size or selected qubits of the subsystem. Please refer to the following chart for the selection.


### Bit select hint

A bit string mapping to the index of qubit is `'76543210'` for 8 qubits example here.

| the input | bitStringRange | String Slice | Actual Qubit Included |
| --------- | -------------- | ------------ | --------------------- |
| `2`       | `(6, 8)`       | `'10'`       | 01                    |
| `4`       | `(4, 8)`       | `'3210'`     | 0123                  |
| `6`       | `(2, 8)`       | `'543210'`   | 012345                |
| `(2, 4)`  | `(2, 4)`       | `'54'`       | 45                    |
| `(4, 6)`  | `(4, 6)`       | `'32'`       | 23                    |
| `(2, 6)`  | `(2, 6)`       | `'5432'`     | 2345                  |
| `(-2, 2)` | `(-2, 2)`      | `'1076'`     | 6701                  |
| `(-4, 2)` | `(-4, 2)`      | `'321076'`   | 670123                |
| `(-2, 4)` | `(-2, 4)`      | `'107654'`   | 456701                |
| `3`       | `(5, 8)`       | `'210'`      | 012                   |
| `(5, 7)`  | `(5, 7)`       | `'21'`       | 12                    |
| `(4, 7)`  | `(4, 7)`       | `'321'`      | 123                   |


In [14]:
exp1_report1 = experiment_randomized.exps[exp1].analyze(degree=(3, 4))
exp1_report2 = experiment_randomized.exps[exp1].analyze(degree=1)

|  - 00:00 < ?

|  - 00:00 < ?

- The `analyze` will return the result of this analysis.


In [15]:
exp1_report2.export()

({'purity': 1.0566659164428711,
  'entropy': -0.0795193153427059,
  'puritySD': 0.48580666272767997,
  'entropySD': 0.6632851994577739,
  'bitStringRange': [7, 8],
  'allSystemSource': "AnalysisHeader(serial=0, datetime='2024-06-03 17:55:47', summoner=None, log={})",
  'purityAllSys': 1.2927798652648925,
  'entropyAllSys': -0.37047663374945394,
  'puritySDAllSys': 1.7144946824753815,
  'entropySDAllSys': 1.9133133509399982,
  'bitsStringRangeAllSys': [3, 4],
  'errorRate': -0.13750956989348648,
  'mitigatedPurity': 0.9302139741551041,
  'mitigatedEntropy': 0.10436548195147936,
  'num_qubits': 8,
  'measure': ['measure range:', [0, 8]],
  'measureActually': [0, 8],
  'measureActuallyAllSys': [0, 8],
  'countsNum': 100,
  'takingTime': 0.00032463,
  'takingTimeAllSys': 0,
  'input': {'degree': 1, 'shots': 1024, 'unitary_loc': [0, 8]},
  'header': {'serial': 1,
   'datetime': '2024-06-03 17:55:47',
   'summoner': None,
   'log': {}}},
 {'purityCells': {5: 1.02880859375,
   99: 0.528038024

In [16]:
print(exp1_report1)
print(exp1_report1.header)

<qurrentRandomized.Analysis with serial=0, AnalysisInput(degree=(3, 4), shots=1024, unitary_loc=(0, 8)), AnalysisContent(purity=1.0482955360412598, entropy=-0.06804549958001889, and others), 0 unused arguments>
AnalysisHeader(serial=0, datetime='2024-06-03 17:55:47', summoner=None, log={})


- It can also be found in the attribute `.reports` of `Experiment` object.


In [17]:
print(experiment_randomized.exps[exp1].reports)
print(experiment_randomized.exps[exp1].reports[0])
print(experiment_randomized.exps[exp1].reports[0].header)

<AnalysisContainer={
    0: ...
    1: ...
} with 2 analysis load, a customized dictionary>
<qurrentRandomized.Analysis with serial=0, AnalysisInput(degree=(3, 4), shots=1024, unitary_loc=(0, 8)), AnalysisContent(purity=1.0482955360412598, entropy=-0.06804549958001889, and others), 0 unused arguments>
AnalysisHeader(serial=0, datetime='2024-06-03 17:55:47', summoner=None, log={})


- We can use `.export()` to get the result of this analysis.


In [18]:
main, side_prodct = experiment_randomized.exps[exp1].reports[0].export()

- And there is it


In [19]:
main

{'purity': 1.0482955360412598,
 'entropy': -0.06804549958001889,
 'puritySD': 0.4938075744472797,
 'entropySD': 0.6795924568169278,
 'bitStringRange': [3, 4],
 'allSystemSource': 'independent',
 'purityAllSys': 1.2927798652648925,
 'entropyAllSys': -0.37047663374945394,
 'puritySDAllSys': 1.7144946824753815,
 'entropySDAllSys': 1.9133133509399982,
 'bitsStringRangeAllSys': [0, 8],
 'errorRate': -0.13750956989348648,
 'mitigatedPurity': 0.923745005045628,
 'mitigatedEntropy': 0.11443343667364543,
 'num_qubits': 8,
 'measure': ['measure range:', [0, 8]],
 'measureActually': [0, 8],
 'measureActuallyAllSys': [0, 8],
 'countsNum': 100,
 'takingTime': 0.000794718,
 'takingTimeAllSys': 0.00318872,
 'input': {'degree': [3, 4], 'shots': 1024, 'unitary_loc': [0, 8]},
 'header': {'serial': 0,
  'datetime': '2024-06-03 17:55:47',
  'summoner': None,
  'log': {}}}

- Also, `sideProdct` will record the data of each circuit, and you can use it to do more analysis.


In [20]:
side_prodct

{'purityCells': {6: 0.5011215209960938,
  26: 1.3175888061523438,
  1: 1.8570919036865234,
  45: 1.490234375,
  88: 0.5002059936523438,
  31: 1.801931381225586,
  75: 0.6236476898193359,
  66: 0.5185909271240234,
  30: 1.3525619506835938,
  87: 1.192962646484375,
  13: 1.229269027709961,
  99: 1.253988265991211,
  51: 1.924795150756836,
  78: 1.9133968353271484,
  38: 1.046346664428711,
  34: 1.747915267944336,
  0: 0.6322021484375,
  95: 0.5087032318115234,
  2: 1.330617904663086,
  32: 0.9812240600585938,
  7: 0.5655117034912109,
  4: 0.5227108001708984,
  8: 0.5413417816162109,
  10: 1.476003646850586,
  33: 0.6027450561523438,
  36: 0.5313339233398438,
  39: 1.7693843841552734,
  41: 0.511077880859375,
  42: 1.553091049194336,
  55: 0.6916255950927734,
  23: 0.6105556488037109,
  84: 0.5313339233398438,
  90: 0.552734375,
  91: 0.515472412109375,
  97: 0.7744350433349609,
  12: 1.669036865234375,
  74: 0.7645015716552734,
  5: 0.5313339233398438,
  17: 0.5375423431396484,
  72: 0.5

In [21]:
experiment_randomized.exps[exp1].reports[0].statesheet()

 # qurrentRandomized.Analysis with serial=0
 - header
   - serial ----------------------- 0
   - datetime --------------------- 2024-06-03 17:55:47
   - summoner --------------------- None
   - log -------------------------- {}
 - input
   - degree ----------------------- (3, 4)
   - shots ------------------------ 1024
   - unitary_loc ------------------ (0, 8)
 - outfields -------------------- 0 ........  # Number of unused arguments.
 - content
   - purity ----------------------- 1.0482955360412598
   - entropy ---------------------- -0.06804549958001889
   - puritySD --------------------- 0.4938075744472797
   - entropySD -------------------- 0.6795924568169278
   - purityCells ------------------ {6: 0.5011215209960938, 26: 1.3175888061523438, 1: 1.8570919036865234, 45: 1.490234375, 88: 0.5002059936523438, 31: 1.801931381225586, 75: 0.6236476898193359, 66: 0.5185909271240234, 30: 1.3525619506835938, 87: 1.192962646484375, 13: 1.229269027709961, 99: 1.253988265991211, 51: 1.924795150

---

## Post-Process Availablities and Version Info

We currently do not support Qsikit 1.0 for we are working on it, and we will support it in the future.


In [22]:
from qurry.process import AVAIBILITY_STATESHEET
from qurry.tools.qiskit_version import QISKIT_VERSION_STATESHEET

print(AVAIBILITY_STATESHEET)
print(QISKIT_VERSION_STATESHEET)

 | Qurry version: 0.7.4.dev1
--------------------------------------------------------
 ### Qurry Post-Processing
   - Backend Availability ......... Python Cython Rust  
 - randomized_measure
   - entangled_core ............... True   True   True  
   - purity_cell .................. True   True   True  
   - wavefunction_overlap ......... True   True   None  
   - echo_cell .................... True   True   None  
 - utils
   - randomized ................... True   True   True  
   - construct .................... True   None   True  
--------------------------------------------------------
   + True ..... Working normally.
   + False .... Exception occurred.
   + None ..... Not supported yet.
--------------------------------------------------------

 | Qurry version: 0.7.4.dev1
--------------------------------------------
 ### Qiskit version
 - main
   - qiskit-aer ................... 0.11.2
   - qiskit-aer-gpu ............... 0.11.2
   - qiskit-ibm-provider .......... 0.11.0
   - q